## Feature extraction Using **tsfresh**

In this secetion we using tsfresh library for features extraction.

**tsfresh** library allow diffent data foramts for feature extraction, our current task is sutibale
with [dictionary with flat dataframe
format](https://tsfresh.readthedocs.io/en/latest/text/data_formats.html#input-option-3-dictionary-of-flat-dataframes)

So first step is to convert our data into suggest above format. Then extact featues form the
dataset. 

For any usuall classsification problem, we need to have some set of features, but in our case we
have different time series, so our first taks is to convert these time series into features. 

Transformations: 
- Lienar transformations using PCA: As taken from different thoughts from
[here](https://www.linkedin.com/advice/0/what-advantages-disadvantages-using-pca), we are ingnoring
exploring PCA on time series.
- Non-Linear transformations like T-SNE: 
  - T-SNE is a technique used for visual the data, it does not learn any function from the original
  space to the new (lower) dimensional one. so this technique is not sutibale for feature
  extraction. Please refer the discussion
  [here](https://stats.stackexchange.com/questions/340175/why-is-t-sne-not-used-as-a-dimensionality-reduction-technique-for-clustering-or)
  - Auto encoders are a good options, we will do this in later sections.
- Latent Space Feaure extractions
  - tsfresh
  - pyts
  - cesium
  

Note: 
- Please be aware, tsfresh feature extraction is a bit slow.
- We are using only default setting for feature extraction, Please refere documentation for details and advanced settings for feature extraction.


Various approches for feature extraction and feature t

In [1]:
import pandas as pd
from tsfresh.feature_extraction import extract_features

from src.hydraulic.config import DATA_PATH, SEPERATOR, PROFILE_COLUMNS, FILES_NAMES, DATA_OUTPUT_PATH

In [2]:
def convert_flat_dataframe(raw_data: pd.DataFrame) -> pd.DataFrame:
    """convert wide time series format into flat data format

    Args:
        raw_data (pd.DataFrame): wide time series dataframe

    Returns:
        pd.DataFrame: flat data froamt dataset
    """
    raw_data_ = (
        raw_data.reset_index()
                .rename(columns= {'index': 'id'})
                .set_index('id')
                .unstack()
                .to_frame('value')
                .reset_index()
                .rename(columns={'level_0':'time'})
    )

    return raw_data_

In [3]:
# Load data and then convert data into flat data format
flat_ts_dict = {}
for sensor_type, data_file_names_list in FILES_NAMES.items():
    for file_name in data_file_names_list:
        ts_data = pd.read_csv(f'./data/{file_name}.txt', sep='\t', header=None)
        flat_data = convert_flat_dataframe(ts_data)
        flat_ts_dict[f"{sensor_type}_{file_name}"] = flat_data

In [4]:
tsfresh_features = extract_features(
    flat_ts_dict, column_id="id", column_sort="time", column_kind=None, column_value="value"
)

tsfresh_features.to_parquet(f"{DATA_OUTPUT_PATH}/tsfresh_features.parquet")

Feature Extraction:  24%|██▍       | 6/25 [44:50:11<141:58:56, 26901.91s/it]
Process SpawnPoolWorker-3:
Traceback (most recent call last):
  File "/Users/sivajidoguparthi/opt/miniconda3/envs/ds_activity/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/sivajidoguparthi/opt/miniconda3/envs/ds_activity/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/sivajidoguparthi/opt/miniconda3/envs/ds_activity/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/Users/sivajidoguparthi/opt/miniconda3/envs/ds_activity/lib/python3.8/site-packages/tsfresh/utilities/distribution.py", line 43, in _function_with_partly_reduce
    results = list(itertools.chain.from_iterable(results))
  File "/Users/sivajidoguparthi/opt/miniconda3/envs/ds_activity/lib/python3.8/site-packages/tsfresh/utilities/distribution.py", line 42, in <genexpr>
  

KeyboardInterrupt: 